<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Paola Silva Muñoz
- Nombre de alumno 2: Carla Guzmán Roa


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/Paito249/MDS7202`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
!wget -O sales.csv https://github.com/Paito249/MDS7202/raw/main/lab_09/sales.csv

--2023-11-20 02:22:57--  https://github.com/Paito249/MDS7202/raw/main/lab_09/sales.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Paito249/MDS7202/main/lab_09/sales.csv [following]
--2023-11-20 02:22:57--  https://raw.githubusercontent.com/Paito249/MDS7202/main/lab_09/sales.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 656341 (641K) [text/plain]
Saving to: ‘sales.csv’

sales.csv           100%[===================>] 640.96K  --.-KB/s    in 0.009s  

2023-11-20 02:22:57 (66.2 MB/s) - ‘sales.csv’ saved [656341/656341]



In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

df_original = pd.read_csv('sales.csv')
df_original['date'] = pd.to_datetime(df_original['date'])

df_original.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib

In [5]:
#Se hace una copia del df_original
df = df_original.copy()

Se separan los conjuntos train (70%), validation (20%) y test (10%).

In [6]:
df_1 = df_original.copy()

#Se separan los conjuntos dentro del código ya que queremos dropear algunas columnas
df_2 = df_1.drop(['id','lat','long'],axis=1)

#Se hace una copia de los datos sin las variables innecesarias
df = df_2.copy()

X_train, X_val_test = train_test_split(df, train_size=0.7, random_state=42)
X_val,X_test = train_test_split(X_val_test, test_size=1/3, random_state=42)

# #Se comprueban los porcentajes
# print(f'El porcentaje del conjunto train es {len(X_train)/len(df)} \nEl porcentaje del conjunto val es {len(X_val)/len(df)} \nEl porcentaje del conjunto test es {len(X_test)/len(df)}')

#Se crea la función que extraerá día, mes y año desde la variable date. La variable ya tiene formato de fecha.
#se usará la librería datetime para extraer directamente.
def separar_fecha(df):
  df['day'] = df['date'].dt.day.astype('category')
  df['month'] = df['date'].dt.month.astype('category')
  df['year'] = df['date'].dt.year.astype('category')
  df.drop(['date'],axis=1,inplace=True)
  return df

#Ahora se crea el FunctionTransformer
info_fecha = FunctionTransformer(func=separar_fecha,validate=False)

#dada la información anterior se clasifican las variables en numéricas o categóricas
cols_num = ['pop','price','day','month','year']
cols_cat = ['city','shop','brand','container','capacity']

#se descartan las variables id (no aporta información al dataset y hay otro id propio del dataset), lat y long
#(tienen correlación directa con shop).

# Se crea el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numericas', StandardScaler(), cols_num),  # Escalado estándar para variables numéricas
        ('categoricas', OneHotEncoder(sparse_output=False), cols_cat)  # OneHotEncoder para variables categóricas
    ],
    remainder='passthrough'
    )
preprocessor.set_output(transform="pandas")

# Separar las variables objetivo (y) y las variables predictoras (X)
y_train = X_train['quantity']
X_train = X_train.drop('quantity', axis=1)
y_val = X_val['quantity']
X_val = X_val.drop('quantity',axis=1)
y_test = X_test['quantity']
X_test = X_test.drop('quantity',axis=1)


#Se crea el pipeline
pipeline = Pipeline([('separar_fecha',info_fecha),
                     ('preprocesamiento',preprocessor),
                     ('dummy_regressor', DummyRegressor(strategy='mean'))
                     ])

pipeline.fit(X_train, y_train)



Pipeline(steps=[('separar_fecha',
                 FunctionTransformer(func=<function separar_fecha at 0x7cc2bd7ceb90>)),
                ('preprocesamiento',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numericas', StandardScaler(),
                                                  ['pop', 'price', 'day',
                                                   'month', 'year']),
                                                 ('categoricas',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container',
                                                   'capacity'])])),
                ('dummy_regressor', DummyRegressor())])

In [7]:
# Preprocesar y predecir en el conjunto de validación
predictions_val = pipeline.predict(X_val)

# Calcular el MAE en los datos de validación
mae_Dummy = mean_absolute_error(y_val, predictions_val)
mae_Dummy

13298.497767341096

In [8]:
df = df_2.copy()

#Se reinician las variables
X_train, X_val_test = train_test_split(df, train_size=0.7, random_state=42)
X_val,X_test = train_test_split(X_val_test, test_size=1/3, random_state=42)

# Separar las variables objetivo (y) y las variables predictoras (X)
y_train = X_train['quantity']
X_train = X_train.drop('quantity', axis=1)
y_val = X_val['quantity']
X_val = X_val.drop('quantity',axis=1)
y_test = X_test['quantity']
X_test = X_test.drop('quantity',axis=1)

#Se crea el pipeline2
pipeline2 = Pipeline([('separar_fecha',info_fecha),
                     ('preprocesamiento',preprocessor),
                     ('XGBRegressor', XGBRegressor(enable_categorical=True))
                     ])

pipeline2.fit(X_train, y_train)

Pipeline(steps=[('separar_fecha',
                 FunctionTransformer(func=<function separar_fecha at 0x7cc2bd7ceb90>)),
                ('preprocesamiento',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numericas', StandardScaler(),
                                                  ['pop', 'price', 'day',
                                                   'month', 'year']),
                                                 ('categoricas',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container',
                                                   'capacity'])])),
                ('XGB...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [9]:
#Preprocesar y predecir con el conjunto de validación
predict2_val= pipeline2.predict(X_val)

# Calcular el MAE en los datos de testing
mae_XGB = mean_absolute_error(y_val, predict2_val)
mae_XGB


####DISMUNYE EL MAE, ES MEJOR ESTE MODELO

2327.4770461964495

In [10]:
predict2_test= pipeline2.predict(X_test)
# Calcular el MAE en los datos de testing
mae_test = mean_absolute_error(y_test, predict2_test)
mae_test

2322.836517415801

In [11]:
# Guardar el modelo DummyRegressor en un archivo .pkl
joblib.dump(pipeline, 'dummy_regressor.pkl')

# Guardar el modelo XGBRegressor en un archivo .pkl
joblib.dump(pipeline2, 'XGBRgressor.pkl')

['XGBRgressor.pkl']

El MAE disminuye notablemente (desde 13298.49 a 2327.47), esto significa que el modelo que utiliza XGBRegressor es mucho mejor que el DummyRegressor.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [12]:
df = df_2.copy()

#Se reinician las variables
X_train, X_val_test = train_test_split(df, train_size=0.7, random_state=42)
X_val,X_test = train_test_split(X_val_test, test_size=1/3, random_state=42)

# Separar las variables objetivo (y) y las variables predictoras (X)
y_train = X_train['quantity']
X_train = X_train.drop('quantity', axis=1)
y_val = X_val['quantity']
X_val = X_val.drop('quantity',axis=1)
y_test = X_test['quantity']
X_test = X_test.drop('quantity',axis=1)

# Define el modelo XGBRegressor con restricciones monótonas
monotone_constraints = {'numericas__price': -1}
xgb_model = XGBRegressor(objective='reg:squarederror', monotone_constraints=monotone_constraints,enable_categorical=True)

#Se crea el pipeline2
pipeline3 = Pipeline([('separar_fecha',info_fecha),
                     ('preprocesamiento',preprocessor),
                     #('XGBRegressor', XGBRegressor())
                     ('XGBRegressor',xgb_model)
                     ])

pipeline3.fit(X_train, y_train)


Pipeline(steps=[('separar_fecha',
                 FunctionTransformer(func=<function separar_fecha at 0x7cc2bd7ceb90>)),
                ('preprocesamiento',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numericas', StandardScaler(),
                                                  ['pop', 'price', 'day',
                                                   'month', 'year']),
                                                 ('categoricas',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container',
                                                   'capacity'])])),
                ('XGB...
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'numericas__price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [13]:
#Preprocesar y predecir con el conjunto de validación
predict3_val= pipeline3.predict(X_val)

# Calcular el MAE en los datos de testing
mae_mon = mean_absolute_error(y_val, predict3_val)
mae_mon



2419.0019158325604

In [14]:
#x = pipeline3.fit_transform(df)
predict3_test= pipeline3.predict(X_test)
# Calcular el MAE en los datos de testing
mae_test = mean_absolute_error(y_test, predict3_test)
mae_test

2400.2623203471903

In [15]:
# Guardar el modelo XGBRegressor en un archivo .pkl
joblib.dump(pipeline3, 'XGBRegressor_monotone.pkl')

['XGBRegressor_monotone.pkl']

Al entrenar el modelo forzando la relación monótoma negativa entre el precio, y la variable objetivo (cantidad), se observa que el modelo empeora levemente su desempeño. Esto tiene sentido puesto que no hay información que haga referencia a que a mayor cantidad de elementos comprados el precio sea menor. De hecho, al revisar la primeras 5 filas del dataset se puede ver un ejemplo claro en que esto no sucede (filas 0 y 3). Entonces, al imponer relación monótona negativa, estamos pasando al modelo información que no se condice con los datos.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Explicación hiperparámetro y su rol en el modelo:


1. **learning_rate:** Indica el tamaño del paso dado en cada iteración del algoritmo. Al multiplicarse por el gradiente de la función de pérdida, un valor más pequeño conlleva pasos más pequeños en el entrenamiento y, por ende, una duración mayor del mismo. Un valor máximo razonable dentro del entrenamiento es 0.1.
2. **n_estimators:** Representa la cantidad de árboles creados en el algoritmo, lo que influye en la complejidad del modelo y en el tiempo de entrenamiento. Los valores enteros son adecuados para este hiperparámetro.
3. **max_depth:** Es la profundidad de cada árbol, correspondiente a la cantidad de preguntas que el modelo puede realizar para predecir. Se considera coherente un rango entre 3 y 10, ya que una mayor profundidad implica un modelo más complejo y un entrenamiento más prolongado.
4. **max_leaves:** Controla el número máximo de hojas en el árbol resultante, regulando la complejidad del modelo de manera directa, similar a max_depth. El rango sugerido es apropiado, aunque se aconseja que sea inferior a 64 para no ralentizar excesivamente el entrenamiento.
5. **min_child_weight:** Establece el peso mínimo necesario para dividir un nodo, influenciando la complejidad del modelo al descartar más nodos con valores mayores. El rango es adecuado para este hiperparámetro, aunque algunos modelos pueden incluso utilizar un valor como 200.
6. **reg_alpha:** Aplica una regularización L1 a los pesos de los nodos, penalizando los valores absolutos de los pesos para hacer el modelo más robusto y evitar la dispersión. El rango está entre 0 y 1, siendo 0 una penalización nula.
7. **reg_lambda:** Implementa una regularización L2 a los pesos de los nodos, penalizando los valores cuadráticos de los pesos para prevenir el sobreajuste al suavizar los pesos que se desvían significativamente. Al igual que reg_alpha, su rango es de 0 a 1, siendo 0 una penalización nula.
8. **min_frequency:** Determina la frecuencia mínima requerida para considerar una categoría en el modelo. Se representa como un porcentaje de la frecuencia total, multiplicando este parámetro por el número de muestras, por lo que se consideracomo un valor flotante entre 0.0 y 1.0.

Según las descripciones dadas, se encuentran oportunos los rangos de optimización entregados.

In [41]:
!pip install --quiet optuna

In [17]:
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
import random

In [42]:
df = df_2.copy()

#Se reinician las variables
X_train, X_val_test = train_test_split(df, train_size=0.7, random_state=42)
X_val,X_test = train_test_split(X_val_test, test_size=1/3, random_state=42)

# Separar las variables objetivo (y) y las variables predictoras (X)
y_train = X_train['quantity']
X_train = X_train.drop('quantity', axis=1)
y_val = X_val['quantity']
X_val = X_val.drop('quantity',axis=1)
y_test = X_test['quantity']
X_test = X_test.drop('quantity',axis=1)


In [43]:
def objective(trial):
    random.seed(42)
    # Espacio de búsqueda para hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    # Espacio de búsqueda para el hiperparámetro de OneHot
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Definición del modelo con hiperparámetros
    model = XGBRegressor(
        learning_rate = learning_rate,
        n_estimators = n_estimators,
        max_depth = max_depth,
        max_leaves = max_leaves,
        min_child_weight = min_child_weight,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        random_state=42)

    preprocessor = ColumnTransformer(
    transformers=[
        ('numericas', StandardScaler(), cols_num),  # Escalado estándar para variables numéricas
        ('categoricas', OneHotEncoder(sparse_output=False,  min_frequency=min_frequency), cols_cat)  # OneHotEncoder para variables categóricas
    ],
    remainder='passthrough'
    )
    pipeline_optuna=Pipeline([('separar_fecha',info_fecha),
                      ('preprocesamiento',preprocessor),
                      ('regressor', model)
                      ])


    X_train_optuna=X_train.copy()

    # Entrenamiento y evaluación del modelo con validación cruzada
    mae = -cross_val_score(pipeline_optuna, X_train_optuna, y_train, cv=3, scoring='neg_mean_absolute_error').mean()

    return mae


#Se utiliza TPESampler como método de muestreo y se fija semilla para reproducibilidad de resultados
study = optuna.create_study(sampler=TPESampler())
study.optimize(objective, n_trials=32, timeout=300)

print(f'Numero de trials: {len(study.trials)}')
print(f'MAE: {study.best_value}')

# Obtención de los mejores hiperparámetros encontrados
best_params = study.best_params
print('Mejores parametros:')
for key, value in best_params.items():
    print(f'    {key}: {value}')

Numero de trials: 32
MAE: 2399.1440295711086
Mejores parametros:
    learning_rate: 0.08189542406804805
    n_estimators: 594
    max_depth: 7
    max_leaves: 75
    min_child_weight: 4
    reg_alpha: 0.6180604018886172
    reg_lambda: 0.8938018223751114
    min_frequency: 0.07626028743935584


In [44]:
mae_op=study.best_value

In [45]:
modelo_entrenado = XGBRegressor(X_train, y_train, best_params,random_state=42)
joblib.dump((modelo_entrenado, best_params), 'modelo_con_mejores_parametros.pkl')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:726: FutureWarning:

Pass `objective` as keyword args.



['modelo_con_mejores_parametros.pkl']

Número de trials: 32
*   MAE: 2399.1440295711086

Mejores parametros:
* learning_rate: 0.08189542406804805
* n_estimators: 594
* max_depth: 7
* max_leaves: 75
* min_child_weight: 4
* reg_alpha: 0.6180604018886172
* reg_lambda: 0.8938018223751114
* min_frequency: 0.07626028743935584

El MAE de RGBRegressor con optimización de parámetros es mejor que el sin optimización. Esto es resultado de que al aplicar la optimización el modelo busca automáticamente los mejores parámetros para hacer el entrenamiento con RGBRegressor.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

Prunning (poda) consiste en un metodo de simplificar modelos de árboles, eliminando nodos o ramas poco relevantes durante o después de la construcción del árbol. Este proceso, ayuda al reducir el sobreajuste, mejorando la capacidad de generalización del modelo para datos nuevos.

Además, al eliminar partes redundantes, se reduce la complejidad del modelo, optimizando el uso de recursos computacionales tanto en el entrenamiento como en la predicción, lo que conduce a modelos más eficientes y precisos.

In [46]:
from optuna.integration import XGBoostPruningCallback

In [47]:
def objective_pruning(trial):
    # Espacio de búsqueda para hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)



    preprocessor = ColumnTransformer(
        transformers=[
            ('numericas', StandardScaler(), cols_num),
            ('categoricas', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), cols_cat)
        ],
        remainder='passthrough'
    )

    pipeline_optuna = Pipeline([
        ('separar_fecha', info_fecha),
        ('preprocesamiento', preprocessor),
    ])

    # Copia de los datos de entrenamiento
    X_train_optuna = X_train.copy()
    X_val_optuna = X_val.copy()

    X_train_optuna_processed = pipeline_optuna.fit_transform(X_train_optuna)
    X_val_optuna_processed = pipeline_optuna.transform(X_val_optuna)

    # Entrenamiento con prunning
    pruning_callback = XGBoostPruningCallback(trial, observation_key="validation_0-mae")

    # Definición del modelo con hiperparámetros
    model = XGBRegressor(
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    max_depth=max_depth,
    max_leaves=max_leaves,
    min_child_weight=min_child_weight,
    reg_alpha=reg_alpha,
    reg_lambda=reg_lambda,
    eval_metric='mae',
    early_stopping_rounds=10,
    callbacks=[pruning_callback],
    random_state=42
    )

    model.fit(
        X_train_optuna_processed, y_train,
        eval_set=[(X_val_optuna_processed, y_val)],
        verbose=False
    )

    # Cálculo del MAE
    y_pred = model.predict(X_val_optuna_processed)
    mae = mean_absolute_error(y_val, y_pred)

    return mae

# Silenciar los prints
optuna.logging.set_verbosity(optuna.logging.WARNING)

#Se utiliza TPESampler como método de muestreo y se fija semilla para reproducibilidad de resultados
study_pruning = optuna.create_study(sampler=TPESampler())
study_pruning.optimize(objective_pruning, n_trials=32, timeout=300, show_progress_bar=True)

print(f'Numero de trials: {len(study_pruning.trials)}')
print(f'MAE: {study_pruning.best_value}')

# Obtención de los mejores hiperparámetros encontrados
best_params = study_pruning.best_params
print('Mejores parametros:')
for key, value in best_params.items():
    print(f'    {key}: {value}')

  0%|          | 0/32 [00:00<?, ?it/s]

Numero de trials: 32
MAE: 2208.6780558022615
Mejores parametros:
    learning_rate: 0.07697433951910847
    n_estimators: 678
    max_depth: 5
    max_leaves: 68
    min_child_weight: 2
    reg_alpha: 0.8422939981597326
    reg_lambda: 0.757270570906535
    min_frequency: 0.10637950893345961


In [48]:
mae_prun = study_pruning.best_value

In [49]:
best_params

{'learning_rate': 0.07697433951910847,
 'n_estimators': 678,
 'max_depth': 5,
 'max_leaves': 68,
 'min_child_weight': 2,
 'reg_alpha': 0.8422939981597326,
 'reg_lambda': 0.757270570906535,
 'min_frequency': 0.10637950893345961}

In [50]:
# Crear el modelo con los mejores hiperparámetros
items = list(best_params.items())
params = {key: value for key, value in items[:-1]}  # Excluir el último elemento de los parámetros

best_model = XGBRegressor(**params, random_state=42)

# Utilizar el preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('numericas', StandardScaler(), cols_num),
        ('categoricas', OneHotEncoder(sparse_output=False, min_frequency=best_params['min_frequency']), cols_cat)
    ],
    remainder='passthrough'
)

pipeline_optuna = Pipeline([
    ('separar_fecha', info_fecha),
    ('preprocesamiento', preprocessor),
])

# Copia de los datos de entrenamiento
X_train_optuna = X_train.copy()
X_val_optuna = X_val.copy()

X_train_optuna_processed = pipeline_optuna.fit_transform(X_train_optuna)
X_val_optuna_processed = pipeline_optuna.transform(X_val_optuna)

# Entrenar el modelo con los datos y guardar el modelo entrenado
best_model.fit(X_train_optuna_processed, y_train)

joblib.dump(best_model, 'modelo_con_mejores_parametros_pruning.pkl')

['modelo_con_mejores_parametros_pruning.pkl']

Numero de trials: 32
MAE: 2208.6780558022615
Mejores parametros:
    learning_rate: 0.07697433951910847
    n_estimators: 678
    max_depth: 5
    max_leaves: 68
    min_child_weight: 2
    reg_alpha: 0.8422939981597326
    reg_lambda: 0.757270570906535
    min_frequency: 0.10637950893345961


Se observa un mejor desempeño que el modelo anterior.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [51]:
!pip install optuna-dashboard
!optuna-dashboard sqlite:///example-study.db

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 1969, in _exec_single_context
    self.dialect.do_execute(
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 922, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: version_info

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 73, in _create_scoped_session
    yield session
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 1043, in _init_version_info_model
    version_info = models.VersionInfoModel.find(session)
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/models.py", line 578, in find
    version_info = session.query(cls).one_or_none()
  File "/usr/local/lib/python3.10/dist-packages/sqlalche

In [52]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

SEED = 42

np.random.seed(SEED)

In [60]:
plot_optimization_history(study_pruning)

Se puede ver una mejoria casi instantanea, ya desde la trial 2 se empiezan a ver resultados estables.

In [59]:
plot_parallel_coordinate(study_pruning)

Se observa una tendencia clara en cuanto al comportamiento de los hiperparametros, siendo el camino más optimo:

Objetive Value valores pequeños cercanos a 2500

Learning_rate valores altos a 0.07722

Max_depth medio 5

Max leaves valores cercanos a 70

Min Child y min frequency bajos

n_estimators y reg alpha altos

In [55]:
plot_param_importances(study_pruning)

El parámetro más relevante es `min_frequency`, ya que impacta en las variables categoricas a través de onehotencoder, lo que luego impactará en los demás parámetros.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [56]:
from IPython.display import display

In [57]:


# Generando tabla resumen
resumen = {'Modelos': ['DummyRegressor', 'XGBRegressor', 'XGBRegressor con constraint', 'XGBRegressor con optimización', 'XGBRegressor con optimización y prunning'],
           "MAE": [mae_Dummy, mae_XGB, mae_mon, mae_op, mae_prun]}

df_resumen = pd.DataFrame(resumen)
display(df_resumen)

,Modelos,MAE
0,DummyRegressor,13298.497767
1,XGBRegressor,2327.477046
2,XGBRegressor con constraint,2419.001916
3,XGBRegressor con optimización,2399.144030
4,XGBRegressor con optimización y prunning,2208.678056


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>